### 결측치 처리
1. 삭제
2. 대표값
3. 근처값
4. 결측 예측 모델

<br>

#### 대표값 대체
특징의 타입이 같은 경우

In [2]:
import pandas as pd
import os

os.chdir(r"C:\Users\Gyu\Desktop\mywork\FastOnline\Part 4. 머신러닝을 위한 필수 전처리\데이터")

In [3]:
df = pd.read_csv("cleveland.csv")
X = df.drop('Output', axis = 1)
Y = df['Output']

In [4]:
from sklearn.model_selection import train_test_split
Train_X, Test_X, Train_Y, Test_Y = train_test_split(X, Y)

In [5]:
# 결측치 확인
Train_X.isnull().sum()

Age         0
Sex         0
Cp          0
Trestbps    0
Chol        0
Fbs         0
Restecg     0
Thalach     0
Exang       0
Oldpeak     0
Slope       0
Ca          2
Thal        1
dtype: int64

In [6]:
# 평균 상관 계수 확인 (주의: 모든 변수가 연속형이므로 가능한 접근)
Train_X.corr().sum() / (len(Train_X.columns) - 1)

# 수치가 높지 않다고 판단 => 특징 간 관계가 크지 않음 => 대표값 대체 활용 가능 판단

Age         0.164648
Sex         0.147743
Cp          0.070750
Trestbps    0.155093
Chol        0.104186
Fbs         0.121638
Restecg     0.170187
Thalach    -0.066403
Exang       0.162758
Oldpeak     0.196489
Slope       0.158824
Ca          0.121066
Thal        0.174753
dtype: float64

In [7]:
# 대표값을 활용한 결측치 대체
from sklearn.impute import SimpleImputer

# SimpleImputer 인스턴스화
SI = SimpleImputer(strategy = 'mean')

# 학습
SI.fit(Train_X)

# sklearn instance의 출력은 ndarray이므로 다시 DataFrame으로 바꿔줌
Train_X = pd.DataFrame(SI.transform(Train_X), columns = Train_X.columns)
Test_X = pd.DataFrame(SI.transform(Test_X), columns = Test_X.columns)

Train_X.isnull().sum()

Age         0
Sex         0
Cp          0
Trestbps    0
Chol        0
Fbs         0
Restecg     0
Thalach     0
Exang       0
Oldpeak     0
Slope       0
Ca          0
Thal        0
dtype: int64

복잡한 케이스: 다른 타입의 특징이 있는 경우

In [12]:
df = pd.read_csv("saheart.csv")
X = df.drop('Chd', axis = 1)
Y = df['Chd']
Train_X, Test_X, Train_Y, Test_Y = train_test_split(X, Y)
Train_X.isnull().sum()

Sbp          0
Tobacco      0
Ldl          0
Adiposity    0
Typea        0
Obesity      8
Alcohol      5
Age          0
Famhist      4
dtype: int64

In [13]:
Train_X.corr().sum() / (len(Train_X.columns) - 1)

Sbp          0.312477
Tobacco      0.232009
Ldl          0.288555
Adiposity    0.439326
Typea        0.117336
Obesity      0.367597
Alcohol      0.205913
Age          0.376832
Famhist      0.265572
dtype: float64

In [14]:
# Famhist: 범주형 변수
# 그 외 변수: 연속형 변수

# 대표값을 평균을 사용할지, 최빈값을 사용할지 결정이 어려움 => 둘 다 사용해야 함
# 따라서 데이터를 분할해야 함

Train_X_cate = Train_X[['Famhist']]
Train_X_cont = Train_X.drop('Famhist', axis = 1)

Test_X_cate = Test_X[['Famhist']]
Test_X_cont = Test_X.drop('Famhist', axis = 1)

SI_mode = SimpleImputer(strategy = 'most_frequent')
SI_mean = SimpleImputer(strategy = 'mean')

# 학습
SI_mode.fit(Train_X_cate)
SI_mean.fit(Train_X_cont)

# sklearn instance의 출력은 ndarray이므로 다시 DataFrame으로 바꿔줌
Train_X_cate = pd.DataFrame(SI_mode.transform(Train_X_cate),
                            columns = Train_X_cate.columns)

Test_X_cate = pd.DataFrame(SI_mode.transform(Test_X_cate),
                           columns = Test_X_cate.columns)

Train_X_cont = pd.DataFrame(SI_mean.transform(Train_X_cont),
                            columns = Train_X_cont.columns)

Test_X_cont = pd.DataFrame(SI_mean.transform(Test_X_cont),
                           columns = Test_X_cont.columns)

# 다시 두 데이터를 이어붙여야 함
Train_X = pd.concat([Train_X_cate, Train_X_cont], axis = 1)
Test_X = pd.concat([Test_X_cate, Test_X_cont], axis = 1)

In [15]:
Train_X.isnull().sum()

Famhist      0
Sbp          0
Tobacco      0
Ldl          0
Adiposity    0
Typea        0
Obesity      0
Alcohol      0
Age          0
dtype: int64

Tip. 이진형 변수와 연속형 변수만 포함된 경우에는 SI_mean만 사용하여 결측치를 평균으로 대체한 뒤에, 이진형 변수에 대해서만 round 처리를 하면 하나의 인스턴스만 활용할 수 있음

<br>

---
### 근처값 대체

- 시계열 데이터에 한정
- ffill : 결측치 이전의 유효한 값으로 대체
- bfill : 결측치 이후



In [17]:
df = pd.read_excel("AirQuality.xlsx")
df = df.fillna(method = 'ffill').fillna(method = 'bfill')
# 새로운 데이터의 가능성 때문에 ffill하고 bfill하기
df.isnull().sum()

Date             0
Time             0
CO(GT)           0
PT08.S1(CO)      0
NMHC(GT)         0
C6H6(GT)         0
PT08.S2(NMHC)    0
NOx(GT)          0
PT08.S3(NOx)     0
NO2(GT)          0
PT08.S4(NO2)     0
PT08.S5(O3)      0
T                0
RH               0
AH               0
dtype: int64

### 결측치 대체 모델
1. 결측이 소수 칼럼에 쏠리면 안 된다.
2. 특징 간에 관계가 존재해야 한다.

시간이 오래 소요되는 단점이 있음.


#### KNNImputer
- 결측이 아닌 값을 사용해 이웃을 구한 뒤, 이웃들의 대표값으로 결측 대체
- n_neighbors : 이웃 수, 너무 적으면 정상적으로 이루어지지 않아 5정도가 적절

In [22]:
df = pd.read_csv("mammographic.csv")
X = df.drop('Output', axis = 1)
Y = df['Output']
Train_X, Test_X, Train_Y, Test_Y = train_test_split(X, Y)

In [23]:
Train_X.isnull().sum() / len(Train_X)

BI-RADS    0.002778
Age        0.004167
Shape      0.033333
Margin     0.052778
Density    0.086111
dtype: float64

In [24]:
Train_X.corr().sum() / len(Train_X.columns)

BI-RADS    0.449088
Age        0.413815
Shape      0.513005
Margin     0.534979
Density    0.256597
dtype: float64

In [20]:
# KNN Imputer 인스턴스화
from sklearn.impute import KNNImputer
KI = KNNImputer(n_neighbors = 5)

# KNN Imputer 학습
KI.fit(Train_X)

# 결측 대체
Train_X = pd.DataFrame(KI.transform(Train_X), columns = Train_X.columns)
Test_X = pd.DataFrame(KI.transform(Test_X), columns = Test_X.columns)

In [21]:
Train_X.isnull().sum() / len(Train_X)

BI-RADS    0.0
Age        0.0
Shape      0.0
Margin     0.0
Density    0.0
dtype: float64

### 범주형 변수 처리
1. 더미화
2. 연속형 변수 치환

In [26]:
df = pd.read_csv("car-good.csv")
X = df.drop('Class', axis = 1)
Y = df['Class']
Train_X, Test_X, Train_Y, Test_Y = train_test_split(X, Y)
Train_Y.value_counts()

negative    632
positive     16
Name: Class, dtype: int64

In [27]:
# 문자 라벨을 숫자로 치환 
Train_Y.replace({"negative":-1, "positive":1}, inplace = True)
Test_Y.replace({"negative":-1, "positive":1}, inplace = True)
Train_X.head()

,Buying,Maint,Doors,Persons,Lug_boot,Safety
509,med,high,3,2,med,high
499,med,high,2,4,med,med
175,vhigh,low,2,4,med,med
353,high,med,3,4,small,high
235,high,vhigh,3,2,small,med


In [28]:
# 자세한 범주형 변수 판별 => 모든 변수가 범주형임을 확인
for col in Train_X.columns:
    print(col, len(Train_X[col].unique()))

Buying 4
Maint 4
Doors 3
Persons 2
Lug_boot 3
Safety 3


In [29]:
Train_X = Train_X.astype(str) # 모든 변수가 범주이므로, 더미화를 위해 전부 string 타입으로 변환

In [34]:
Train_df = pd.concat([Train_X, Train_Y], axis = 1)
for col in Train_X.columns: # 보통은 범주 변수만 순회
    temp_dict = Train_df.groupby(col)['Class'].mean().to_dict() # col에 따른 Class의 평균을 나타내는 사전 (replace를 쓰기 위해, 사전으로 만듦)
    Train_df[col] = Train_df[col].replace(temp_dict) # 변수 치환    
    Test_X[col] = Test_X[col].astype(str).replace(temp_dict)

C:\Users\Gyu\Anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [36]:
Train_df.head()

,Buying,Maint,Doors,Persons,Lug_boot,Safety,Class
509,-0.959459,-1.000000,-0.944186,-1.000000,-0.944700,-0.913876,-1
499,-0.959459,-1.000000,-0.962791,-0.903323,-0.944700,-0.936652,-1
175,-1.000000,-0.891566,-0.962791,-0.903323,-0.944700,-0.936652,-1
353,-1.000000,-0.914634,-0.944186,-0.903323,-0.952381,-0.913876,-1
235,-1.000000,-1.000000,-0.944186,-1.000000,-0.952381,-0.936652,-1


In [39]:
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.metrics import f1_score
Train_X = Train_df.drop('Class', axis = 1)
Train_Y = Train_df['Class']

# 치환한 뒤의 모델 테스트
model = KNN().fit(Train_X, Train_Y)
pred_Y = model.predict(Test_X)

f1_score(Test_Y, pred_Y) #ㅎ....

0.0